In [1]:
import requests
import pandas as pd

BASE_URL = "https://fantasy.premierleague.com/api"

bootstrap = requests.get(f"{BASE_URL}/bootstrap-static/").json()

players = {p["id"]: p for p in bootstrap["elements"]}
teams_lookup = {t["id"]: t["name"] for t in bootstrap["teams"]}

In [2]:
def get_formation(picks):
    """
    Determine FPL team formation from picks.
    Only consider the starting 11 (position 1–11).
    """
    formation = {"DEF": 0, "MID": 0, "FWD": 0}

    for pick in picks["picks"]:
        if pick["position"] <= 11:  # starters only - to counter bboost
            if pick["element_type"] == 2:  # DEF
                formation["DEF"] += 1
            elif pick["element_type"] == 3:  # MID
                formation["MID"] += 1
            elif pick["element_type"] == 4:  # FWD
                formation["FWD"] += 1

    return f"{formation['DEF']}-{formation['MID']}-{formation['FWD']}"

def get_full_league_standings(league_id: int):
    standings = []
    page = 1
    while True:
        url = f"{BASE_URL}/leagues-classic/{league_id}/standings/?page_standings={page}"
        resp = requests.get(url).json()
        
        results = resp["standings"]["results"]
        standings.extend(results)
        
        if resp["standings"]["has_next"]:
            page += 1
        else:
            break
    
    return standings

Form the teams table

In [3]:
league_id = 815838
gw = 1

# Fetch league standings
league = get_full_league_standings(league_id)
entries = [team["entry"] for team in league]

teams_data = []
all_picks = []

for entry_id in entries:
    entry_info = requests.get(f"{BASE_URL}/entry/{entry_id}/").json()
    picks = requests.get(f"{BASE_URL}/entry/{entry_id}/event/{gw}/picks/").json()
    all_picks.append(picks)
    transfers = requests.get(f"{BASE_URL}/entry/{entry_id}/transfers/").json()
    
    # Transfers of recent GW
    gw_transfers = [
        (players[t["element_in"]]["web_name"], 
         players[t["element_out"]]["web_name"], 
         players[t["element_in"]]["total_points"] - players[t["element_out"]]["total_points"])
        for t in transfers if t["event"] == gw
    ]
    
    # Formation
    formation = get_formation(picks)
    
    # Captain & VC
    captain = players[next(p["element"] for p in picks["picks"] if p["is_captain"])]["web_name"]
    vice_captain = players[next(p["element"] for p in picks["picks"] if p["is_vice_captain"])]["web_name"]

    playing_XI = ", ".join(players[p["element"]]["web_name"] 
                           for p in picks['picks'] if p["multiplier"] != 0)
    bench = ", ".join(players[p["element"]]["web_name"] 
                           for p in picks['picks'] if p["multiplier"] == 0)
    
    teams_data.append({
        "Team name": entry_info["name"],
        "Total points": entry_info["summary_overall_points"],
        "GW points": entry_info["summary_event_points"],
        "Transfers": gw_transfers,
        "Formation": formation,
        "Captain": captain,
        "Vice Captain": vice_captain,
        "Playing XI": playing_XI,
        "Bench": bench,
        "Chips used": picks.get("active_chip")
    })

df_teams = pd.DataFrame(teams_data)

In [4]:
df_teams

,Team name,Total points,GW points,Transfers,Formation,Captain,Vice Captain,Playing XI,Bench,Chips used
0,Hustle & Flo,78,78,[],3-4-3,Haaland,Wirtz,"Raya, Andersen, Aït-Nouri, Muñoz, Wirtz, Palme...","Areola, Baleba, Alex Moreno, Tuanzebe",None
1,K8 the Gr8,77,77,[],4-4-2,M.Salah,Wood,"Pickford, Schär, Gabriel, Tarkowski, Aina, Cun...","Verbruggen, Ndiaye, Alex Moreno, Kostoulas",None
2,IwobiKlose,60,60,[],3-5-2,M.Salah,Wirtz,"Verbruggen, Frimpong, Murillo, Tarkowski, Wirt...","Dúbravka, Piroe, Gudmundsson, Reinildo",None
3,TheatreOfMemes,55,55,[],4-4-2,M.Salah,Palmer,"Sánchez, Van de Ven, Wan-Bissaka, Konsa, Frimp...","Dúbravka, Estève, A.Ramsey, Marc Guiu",None
4,Saka Potatoes,53,53,[],4-4-2,Delap,Pickford,"Pickford, James, Van de Ven, Wan-Bissaka, Rome...",,bboost
5,All Change,51,51,[],3-4-3,Palmer,João Pedro,"Sánchez, J.Timber, Frimpong, Pedro Porro, Palm...","Dúbravka, Konsa, Malen, Wan-Bissaka",None
6,EZe-pass,49,49,[],4-4-2,Watkins,Palmer,"Dúbravka, Murillo, Tarkowski, Virgil, Pedro Po...","Areola, Estève, Rice, Barnes",None
7,Gameofthrowins,49,49,[],4-4-2,M.Salah,Palmer,"Sánchez, Virgil, Wan-Bissaka, Murillo, Tarkows...","Dúbravka, Hill, Marc Guiu, G.Rodriguez",None
8,Deutschmeister,48,48,[],3-5-2,M.Salah,Palmer,"Sánchez, Murillo, Pedro Porro, Andersen, M.Sal...","Dúbravka, Dorgu, Estève, Marc Guiu",None
9,Elland Roadie,44,44,[],4-4-2,Bowen,Aït-Nouri,"Sels, Cucurella, Kerkez, Pedro Porro, Aït-Nour...","Sánchez, J.Timber, Cunha, Raúl",None


Form the footballers table

In [5]:
from collections import defaultdict

picks_count = defaultdict(int)
captain_count = defaultdict(int)

for team in all_picks:
    active_picks = {p["element"]: p for p in team["picks"]}

    # Determine who was captain + vice
    cap_id = next(p["element"] for p in team["picks"] if p["is_captain"])
    vice_id = next(p["element"] for p in team["picks"] if p["is_vice_captain"])

    # Fetch autosubs info (to check if captain missed out)
    autosubs = team.get("automatic_subs", [])

    # Count all picks
    for p in active_picks.values():
        picks_count[p["element"]] += 1

    # If captain is autosubbed out, vice takes over
    cap_out = any(s["element_out"] == cap_id for s in autosubs)
    if cap_out:
        vice_out = any(s["element_out"] == vice_id for s in autosubs)
        if vice_out: #Case where both C and VC don't play
            None
        else:
            captain_count[vice_id] += 1
    else:
        captain_count[cap_id] += 1

# ===== 5. Build DataFrame =====
footballers_data = []
for p in bootstrap["elements"]:
    footballers_data.append({
        "Player ID":p["id"] ,
        "Footballer name": p["web_name"],
        "Total points": p["total_points"],
        "GW points": p["event_points"],
        "Real team name": teams_lookup[p["team"]],
        "Real team ID": p["team"],
        "Price (in Millions £)": p["now_cost"] / 10,
        "Price last GW (in Millions £)": p["cost_change_event"] / 10 + (p["now_cost"] / 10),
        "Price difference (in Millions £)": p["cost_change_event"] / 10,
        "Times chosen in squad": picks_count[p["id"]],
        "Times captained": captain_count[p["id"]]
    })

df_footballers = pd.DataFrame(footballers_data)

In [6]:
df_footballers.iloc[640:690]

,Player ID,Footballer name,Total points,GW points,Real team name,Real team ID,Price (in Millions £),Price last GW (in Millions £),Price difference (in Millions £),Times chosen in squad,Times captained
640,608,Scarles,0,0,West Ham,19,4.5,4.5,0.0,0,0
641,609,Todibo,1,1,West Ham,19,4.5,4.5,0.0,0,0
642,610,Wan-Bissaka,1,1,West Ham,19,4.5,4.5,0.0,5,0
643,611,Casey,0,0,West Ham,19,4.0,4.0,0.0,0,0
644,612,L.Paquetá,2,2,West Ham,19,6.0,6.0,0.0,0,0
645,613,Souček,1,1,West Ham,19,6.0,6.0,0.0,0,0
646,614,Ward-Prowse,2,2,West Ham,19,6.0,6.0,0.0,0,0
647,615,Summerville,0,0,West Ham,19,5.5,5.5,0.0,0,0
648,616,Álvarez,0,0,West Ham,19,5.0,5.0,0.0,0,0
649,617,Cornet,0,0,West Ham,19,5.0,5.0,0.0,0,0


------------ Metrics -----------

a) Rules based

Helper functions

In [32]:
def all_extremes(series, metric="max", use_abs=False):
    values = series.abs() if use_abs else series
    
    if metric == "max":
        extreme_value = values.max()
    elif metric == "min":
        extreme_value = values.min()
    else:
        raise ValueError("metric must be 'max' or 'min'")
    
    return series[values == extreme_value].index.tolist()

def names_from_indices(df, indices, column="Team name"):
    return [df.iloc[i][column] for i in indices if i < len(df)]

def value_from_first_index(df, indices, column):
    if indices:
        return df.iloc[indices[0]][column]
    return None

Logic

In [33]:
rule_based_metrics = []

# 1 League topper
idx_league_topper = all_extremes(df_teams["Total points"], "max")
all_league_toppers = names_from_indices(df_teams, idx_league_topper, "Team name")
rule_based_metrics.append(f"{all_league_toppers} topped the league")

# 2 League bottom
idx_league_bottom = all_extremes(df_teams["Total points"], "min")
all_league_bottoms = names_from_indices(df_teams, idx_league_bottom, "Team name")
rule_based_metrics.append(f"{all_league_bottoms} are at the bottom of the league")

# 4 Max change in GW points (absolute)
idx_max_change = all_extremes(df_teams["GW points"], "max", use_abs=True)
all_teams_max_change = names_from_indices(df_teams, idx_max_change, "Team name")
value_max_change = value_from_first_index(df_teams, idx_max_change, "GW points")

if value_max_change is not None:
    for idx in idx_max_change:
        total_points = int(df_teams.iloc[idx]["Total points"])
        prev_points = total_points - int(df_teams.iloc[idx]["GW points"])
        max_change_in_points = (
            f"{all_teams_max_change} showed the maximum change of points over the last week. "
            f"Their change: {value_max_change} points from {prev_points} to {total_points}."
        )
        rule_based_metrics.append(max_change_in_points)

# 5 Most points gained
idx_most_points_gained = all_extremes(df_teams["GW points"], "max")
all_teams_most_points_gained = names_from_indices(df_teams, idx_most_points_gained, "Team name")
value_most_points_gained = value_from_first_index(df_teams, idx_most_points_gained, "GW points")
if value_most_points_gained is not None:
    rule_based_metrics.append(
        f"{all_teams_most_points_gained} gained the most points ({value_most_points_gained}) this Gameweek."
    )

# 6 Least points gained
idx_least_points_gained = all_extremes(df_teams["GW points"], "min")
all_teams_least_points_gained = names_from_indices(df_teams, idx_least_points_gained, "Team name")
value_least_points_gained = value_from_first_index(df_teams, idx_least_points_gained, "GW points")
if value_least_points_gained is not None:
    rule_based_metrics.append(
        f"{all_teams_least_points_gained} gained the least points ({value_least_points_gained}) this Gameweek."
    )

# 7 Highest scoring footballer
idx_highest_scoring_footballer = all_extremes(df_footballers["GW points"], "max")
all_highest_scoring_footballers = names_from_indices(df_footballers, idx_highest_scoring_footballer, "Footballer name")
value_highest_scoring_footballers = value_from_first_index(df_footballers, idx_highest_scoring_footballer, "GW points")
if value_highest_scoring_footballers is not None:
    rule_based_metrics.append(
        f"{all_highest_scoring_footballers} gained the most points ({value_highest_scoring_footballers}) this Gameweek."
    )

# 8 Lowest scoring footballer
idx_lowest_scoring_footballer = all_extremes(df_footballers["GW points"], "min")
all_lowest_scoring_footballers = names_from_indices(df_footballers, idx_lowest_scoring_footballer, "Footballer name")
value_lowest_scoring_footballers = value_from_first_index(df_footballers, idx_lowest_scoring_footballer, "GW points")
if value_lowest_scoring_footballers is not None:
    rule_based_metrics.append(
        f"{all_lowest_scoring_footballers} gained the least points ({value_lowest_scoring_footballers}) this Gameweek."
    )

# 9 Highest scoring real team
team_scores = {
    team_id: df_footballers.loc[df_footballers["Real team ID"] == team_id, "GW points"].sum()
    for team_id in range(1, 21)
}
if team_scores:
    max_score = max(team_scores.values())
    max_teams = [tid for tid, score in team_scores.items() if score == max_score]
    highest_scoring_real_teams_list = [teams_lookup[tid] for tid in max_teams]
    rule_based_metrics.append(f"{highest_scoring_real_teams_list} scored the most points ({max_score}).")

# 10 Lowest scoring real team
if team_scores:
    min_score = min(team_scores.values())
    min_teams = [tid for tid, score in team_scores.items() if score == min_score]
    lowest_scoring_real_teams_list = [teams_lookup[tid] for tid in min_teams]
    rule_based_metrics.append(f"{lowest_scoring_real_teams_list} scored the least points ({min_score}).")

# 11 Best/worst transfers
max_value, min_value = float("-inf"), float("inf")
max_transfers, min_transfers = [], []

for idx, row in df_teams.iterrows():
    for tup in row.get("Transfers", []):
        x = tup[2]
        if x > max_value:
            max_value, max_transfers = x, [(idx, tup)]
        elif x == max_value:
            max_transfers.append((idx, tup))
        if x < min_value:
            min_value, min_transfers = x, [(idx, tup)]
        elif x == min_value:
            min_transfers.append((idx, tup))

for idx, tup in max_transfers:
    team_name = df_teams.iloc[idx]["Team name"]
    rule_based_metrics.append(
        f"{team_name} got {tup[0]} in and removed {tup[1]} smartly and saw a change of {max_value} points."
    )

for idx, tup in min_transfers:
    team_name = df_teams.iloc[idx]["Team name"]
    rule_based_metrics.append(
        f"{team_name} got {tup[0]} in and removed {tup[1]} unwisely and saw a change of {min_value} points."
    )

# 13 Highest price increase
idx_highest_increased_price = all_extremes(df_footballers["Price difference (in Millions £)"], "max")
all_highest_price_increase_footballers = [
    df_footballers.iloc[i]["Footballer name"]
    for i in idx_highest_increased_price
    if df_footballers.iloc[i]["Price difference (in Millions £)"] > 0
]
value_highest_increased_price = value_from_first_index(df_footballers, idx_highest_increased_price, "Price difference (in Millions £)")
if all_highest_price_increase_footballers and value_highest_increased_price is not None:
    rule_based_metrics.append(
        f"{all_highest_price_increase_footballers} had the highest price increase (£{value_highest_increased_price}M) this Gameweek."
    )

# 14 Highest priced players vs points
most_expensive_players = df_footballers.sort_values("Price (in Millions £)", ascending=False).head(5)
price_vs_points = "Highest priced footballers \n"
for _, row in most_expensive_players.iterrows():
    if players.get(row["Player ID"], {}).get("minutes", 0) == 0:
        continue
    price_vs_points += f"{row['Footballer name']} valued at {row['Price (in Millions £)']} scored {row['GW points']} points. \n"
rule_based_metrics.append(price_vs_points.strip())

# 15 Most chosen players vs points
most_chosen_players = df_footballers.sort_values("Times chosen in squad", ascending=False).head(5)
chosen_vs_points = "Most picked footballers \n"
for _, row in most_chosen_players.iterrows():
    if players.get(row["Player ID"], {}).get("minutes", 0) == 0:
        continue
    chosen_vs_points += f"{row['Footballer name']} chosen {row['Times chosen in squad']} times scored {row['GW points']} points. \n"
rule_based_metrics.append(chosen_vs_points.strip())

# 16 Most captained players vs points
most_captained_players = df_footballers.sort_values("Times captained", ascending=False).head(5)
captained_vs_points = "Most times picked as captains \n"
for _, row in most_captained_players.iterrows():
    if players.get(row["Player ID"], {}).get("minutes", 0) == 0:
        continue
    captained_vs_points += f"{row['Footballer name']} chosen captain {row['Times captained']} times scored {row['GW points']} points. \n"
rule_based_metrics.append(captained_vs_points.strip())

# 18 Chips usage
chips_usage = []
for _, row in df_teams.iterrows():
    if row.get("Chips used"):
        chips_usage.append(f"{row['Team name']} used the chip(s): {row['Chips used']}")
if chips_usage:
    rule_based_metrics.append(" ".join(chips_usage))


In [34]:
rule_based_metrics

["['Hustle & Flo'] topped the league",
 "['People on the pitch', 'Ha-Cunha Mateta'] are at the bottom of the league",
 "['Hustle & Flo'] showed the maximum change of points over the last week. Their change: 78 points from 0 to 78.",
 "['Hustle & Flo'] gained the most points (78) this Gameweek.",
 "['People on the pitch', 'Ha-Cunha Mateta'] gained the least points (40) this Gameweek.",
 "['Ballard'] gained the most points (17) this Gameweek.",
 "['Doherty'] gained the least points (-1) this Gameweek.",
 "['Man City'] scored the most points (86).",
 "['West Ham', 'Wolves'] scored the least points (18).",
 "['Semenyo', 'Ekitiké', 'Reijnders', 'Wood', 'Ballard'] had the highest price increase (£0.1M) this Gameweek.",
 'Highest priced footballers \nM.Salah valued at 14.5 scored 8 points. \nHaaland valued at 14.0 scored 13 points. \nPalmer valued at 10.5 scored 3 points. \nSaka valued at 10.0 scored 3 points.',
 'Most picked footballers \nPalmer chosen 11 times scored 3 points. \nDúbravka ch

b) LLM based

In [11]:
from dotenv import load_dotenv
import os
from openai import OpenAI

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

In [12]:
rule_based_metrics_text = "\n".join(f"- {metric}" for metric in rule_based_metrics)
df_teams_text = df_teams.to_string(index=False)

In [35]:
prompt = f"""You have to write a commissioner-style roundup commentary for our Fantasy Premier League private league Gameweek 1. I am 
providing you some highlight-metrics that I could find from my data. I am also giving you the standings table of my league which 
you need to observe and make comments about. You also need to have knowledge of the Premier Leagure footballing world and make 
remarks that the fans can relate to, as a part of your sense of humor. 

Here are some of the highlights: {rule_based_metrics_text}
Mention most of these stats in the summary, depending on how interesting each is. Remember to mention table topper, last position holder, highest scoring footballer though.

Here is a teams table. You must use it to comment on the structure of the standings table, and how competitive what part of the table is: {df_teams_text}
Using the table, also look for unusual stuff, or blunders that the players might have made for example not setting up a captain 
or vice-captain, playing injured/suspended players or the ones in transfer talks and not available for selection.

Use the metrics and the table provided to generate a commissioner styled FPL roundup for this GameWeek. Feel free to be creative!
"""

In [36]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a witty fantasy football commissioner."},
        {"role": "user", "content": prompt}
    ],
    temperature=0.7
)

print(response.choices[0].message.content)

Ladies and gentlemen, fantasy football fanatics, and those who accidentally signed up thinking this was a cooking class, welcome to the Gameweek 1 roundup of our Fantasy Premier League private league! It's been a week of highs, lows, and some questionable decisions that would make even Harry Maguire blush.

### **Top of the Table**
Leading the charge with a commanding 78 points, we have the master tactician, **'Hustle & Flo'**. They sprinted from zero to the top faster than Mauricio Pochettino can say "rebuilding phase." With Haaland as captain, their strategy was as solid as a Manchester City defense (on a good day). Kudos on cashing in on Erling's 13-point haul, proving once again that he's not just a Norwegian forest fairy tale.

### **Bottom of the Barrel**
On the other end of the spectrum, we have **'People on the pitch'** and **'Ha-Cunha Mateta'** both sitting at 40 points. They say misery loves company, but this isn't quite the midfield duo anyone was hoping for. Much like West 